# Models

## Setup

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor


In [32]:
concrete = pd.read_csv('data/concrete.csv')
co2 = pd.read_csv('data/co2.csv')

concrete['exp_age'] = np.exp(-.038 * concrete['age'])

concrete['co2_lower'] = sum([concrete[col] * co2.loc[co2.ingredient == col, 'lower_bound'].values[0] for col in concrete.columns[:7]])
concrete['co2_upper'] = sum([concrete[col] * co2.loc[co2.ingredient == col, 'upper_bound'].values[0] for col in concrete.columns[:7]])

concrete = concrete[concrete['age'] < 120]

concrete_train, concrete_test = train_test_split(concrete,
                                                 shuffle=True,
                                                 random_state=487)

features = ['cement', 'slag', 'water', 'superplastic', 'coarseagg', 'fineagg', 'exp_age']

In [4]:
# This function was modified from stackexchange user hughdbrown 
# at this link, 
# https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset

# This returns the power set of a set minus the empty set
def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]


## Linear Model

In [24]:
# for future use, this function gets mean squared error without constantly copy-pasting

def get_slr_mses(data, features_list, y, n_splits=5, rs=97):
    # data is the dataframe
    # features_list is a list of all lists of features we wish to compare
    # eg [[], ['feature1'], ['feature1', 'feature2, 'feature5']]
    # if one list is [], then we make a baseline prediction
    # y is the y feature we are predicting
    # k is the number of cross-validation splits
    # rs is the random_state for kfold
    kfold = KFold(n_splits,
              shuffle=True,
              random_state=rs)
    mses=np.zeros((n_splits, len(features_list)))

    i = 0
    # cross-validation
    for train_index, test_index in kfold.split(data):
        data_tt = data.iloc[train_index]
        data_ho = data.iloc[test_index]

        j = 0
        for features in features_list:
            if features == []:
                # baseline prediction
                pred = data_tt[y].values.mean() * np.ones(len(data_ho))
            else:
                reg = LinearRegression(copy_X=True)
                reg.fit(data_tt[features], data_tt[y])
                pred = reg.predict(data_ho[features])
            
            mses[i, j] = mean_squared_error(y_true=data_ho[y],
                                            y_pred=pred)
            j += 1
        
        i += 1

    return np.mean(mses, axis=0)



We attempt to improve the linear model a little by picking the exponential factor with all features in consideration

In [17]:
features2 = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg',
       'fineagg', 'exp_age']


In [19]:
features2 = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg',
       'fineagg', 'exp_age']

for factor in np.arange(.01, .1, .01):
    concrete_train['exp_age'] = np.exp(-factor * concrete_train['age'])
    print(factor, np.min(get_slr_mses(concrete_train, powerset(features2), 'strength')))

0.01 57.70330104192804
0.02 51.14081744397187
0.03 47.781495017706575
0.04 47.158922486225194
0.05 48.18470524244171
0.060000000000000005 50.0325012084111
0.06999999999999999 52.172666716534025
0.08 54.32070776381877
0.09 56.351408732591054


In [18]:
best_mses = []

for factor in np.arange(.03, .05, .001):
    concrete_train['exp_age'] = np.exp(-factor * concrete_train['age'])
    best_mses.append(np.min(get_slr_mses(concrete_train, powerset(features2), 'strength')))


print(np.arange(.03, .05, .001)[np.argmin(best_mses)], np.min(best_mses))

0.038000000000000006 47.11634037831618


In [5]:
concrete_train['exp_age'] = np.exp(-.038 * concrete_train['age'])

## Random Forest

In [152]:
max_depths = [45]
features = ['cement', 'slag', 'water', 'superplastic', 'coarseagg', 'fineagg', 'exp_age']
n_trees = [100, 200, 300]
n_jobs = [1]
max_features = [None]


grid_cv = GridSearchCV(RandomForestRegressor(), 
                          param_grid = {'max_depth':max_depths, 
                                        'n_estimators':n_trees,
                                        'n_jobs': n_jobs,
                                        'max_features': max_features}, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 5) 

## you fit it just like a model
grid_cv.fit(concrete_train[features], concrete_train['strength'])

print(grid_cv.best_params_)
print(-grid_cv.best_score_)

{'max_depth': 45, 'max_features': None, 'n_estimators': 200, 'n_jobs': 1}
29.750892287963836


In [153]:
max_depths = range(40, 51, 1)
features = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age', 'exp_age']
n_trees = [100, 500]

grid_cv = GridSearchCV(RandomForestRegressor(), 
                          param_grid = {'max_depth':max_depths, 
                                        'n_estimators':n_trees}, 
                          scoring = 'neg_mean_squared_error',
                          cv = 5) 

## you fit it just like a model
grid_cv.fit(concrete_train[features], concrete_train['strength'])

print(grid_cv.best_params_)
print(-grid_cv.best_score_)

{'max_depth': 46, 'n_estimators': 100}
29.206099330106305


## XGBoost

In [155]:
max_depths = range(1, 6)
features = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age', 'exp_age']
n_trees = [400, 500, 600]
# boosters = ['gbtree', 'gblinear', 'dart']
tree_methods = ['exact', 'approx', 'hist']

grid_cv = GridSearchCV(XGBRegressor(learning_rate=.1,
                                ), 
                          param_grid = {'tree_method': tree_methods,
                                        'max_depth':max_depths, 
                                        'n_estimators':n_trees,
                                    #     'booster': boosters
                                        }, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 5)

## you fit it just like a model
grid_cv.fit(concrete_train[features], concrete_train['strength']
                   )
print(grid_cv.best_params_)
print(-grid_cv.best_score_)

{'max_depth': 4, 'n_estimators': 600, 'tree_method': 'exact'}
21.418870786272343


## Histogram-based Gradient Boosting

In [161]:
max_depths = [2, 5, 10]
max_iters = [600, 700, 800]
features = ['cement', 'slag', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age']
learning_rates = [.1]

grid_cv = GridSearchCV(HistGradientBoostingRegressor(loss='squared_error',
                                                     learning_rate=.1,
                                ), 
                          param_grid = {'max_depth':max_depths,
                                        'max_iter': max_iters,
                                        'learning_rate': learning_rates,
                                    #     'booster': boosters
                                        }, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 5) 

## you fit it just like a model
grid_cv.fit(concrete_train[features], concrete_train['strength']
                   )
print(grid_cv.best_params_)
print(-grid_cv.best_score_)

{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 800}
21.90660451714236


This is the best I have gotten it, after much tinkering

## Neural Network

In [8]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler(copy=True)

In [76]:
X_train, y_train = scale.fit_transform(concrete_train[features]), concrete_train['strength']

In [138]:
concrete_tt, concrete_val = train_test_split(concrete_train,
                                             shuffle=True,
                                             random_state=453)

X_tt, y_tt = scale.fit_transform(concrete_tt[features]), concrete_tt['strength']
X_val, y_val = scale.fit_transform(concrete_val[features]), concrete_val['strength']

In [148]:
hidden_layer_sizes = [(200, 200), (250, 250)]
learning_rates = ['constant', 'invscaling', 'adaptive']
solvers = ['sgd']
activations = ['relu']
alphas = [.0001]

grid_cv = GridSearchCV(MLPRegressor(max_iter=10000), 
                          param_grid = {'hidden_layer_sizes': hidden_layer_sizes,
                                        'learning_rate': learning_rates,
                                        'solver': solvers,
                                        'activation': activations,
                                        'alpha': alphas
                                        }, 
                          scoring = 'neg_mean_squared_error', 
                          cv = 5) 


grid_cv.fit(X_train, y_train)

print(grid_cv.best_params_)
print(-grid_cv.best_score_)

{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 200), 'learning_rate': 'adaptive', 'solver': 'sgd'}
29.370374849389577


In [109]:
## Import the following
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import keras_tuner

In [89]:
model1 = keras.Sequential()
model1.add(layers.Dense(200, activation='relu', input_shape=(X_tt.shape[1],)))
model1.add(layers.Dense(100, activation='relu'))
model1.add(layers.Dense(1, activation='relu'))

model1.compile(optimizer = 'rmsprop',
                 loss = 'mean_squared_error',
                 metrics = ['mse'])

In [140]:
model1 = keras.Sequential()
model1.add(layers.Dense(100, activation='relu', input_shape=(X_tt.shape[1],)))
model1.add(layers.Dense(100, activation='relu'))
model1.add(layers.Dense(100, activation='relu'))
model1.add(layers.Dense(100, activation='relu'))
model1.add(layers.Dense(1, activation='relu'))

model1.compile(optimizer = 'rmsprop',
                 loss = 'mean_squared_error',
                 metrics = ['mse'])
n_epochs = 500
history1 = model1.fit(X_tt,
                       y_tt,
                       epochs = n_epochs,
                       batch_size = 50,
                       verbose=0,
                       validation_data = (X_val, 
                                          y_val))

np.min(history1.history['val_mse'])

28.37841033935547